# QKD and Superdense coding

In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import *
from ibm_quantum_widgets import *
from numpy.random import *


# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

qiskit_runtime_service.__init__:INFO:2024-01-31 17:08:14,912: Default instance: ibm-q/open/main


## Quantum Key Distribution (BB84)

**- Step 1**

Alice chooses a string of random bits, e.g.:
1000 1010 1101 0100

And a random choice of bases for each bit:
++X+ XXX+ X+XX XXXX

Alice keeps these two pieces of information private to herself.


**- Step 2**

Alice then encodes each bit onto a string of qubits using the corresponding basis she chose; this means each qubit is in one of the states $|0\rangle$, $|1\rangle$, $|+\rangle$ or $|-\rangle$, chosen at random. In this case, the string of qubits would look like this:
$$ |1\rangle |0\rangle  |+\rangle |0\rangle |-\rangle |+\rangle |-\rangle |0\rangle |-\rangle|1\rangle|+\rangle|- \rangle|+\rangle|-\rangle|+\rangle|+\rangle
This is the message she sends to Bob.


**- Step 3**

Bob then measures each qubit at random, for example, he might use the bases: XZZZ XZXZ XZXZ ZZXZ

And Bob keeps the measurement results private.


**- Step 4**

Bob and Alice then publicly share which basis they used for each qubit. If Bob measured a qubit in the same basis Alice prepared it in, they use this to form part of their shared secret key, otherwise they discard the information for that bit.


**- Step 5**

Finally, Bob and Alice share a random sample of their keys, and if the samples match, they can be sure (to a small margin of error) that their transmission is successful.

Let us implement the protocol, first without Eve's interference. We commence with **step 1**.

In [2]:
seed(seed=0) #to be able to reproduce the results
n = 100 # the number of random bits to be generated... What is the expeted size of the secret key being shared between Alce and Bob?

## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)
print(alice_bits)
# Alice generates bases
# How can we encode the basis choice?
alice_bases = randint(2,size=n)
print(alice_bases)

[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0]
[1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0]


For **step 2**, Let us now define a function to encode a list of bits into the corresponding list of given bases. The output shall be a list of quantum circuits, each one creating the appropiate qubit.

In [3]:
def encode(bitlist,baseslist):
    output = []
    for i in range(n):
        qc = QuantumCircuit(1,1) # we set a  
         # we must use basis +
        if bitlist[i]==1:
            qc.x(0)
        if baseslist[i]==1:
            qc.h(0)
        qc.barrier() #in case we want to display
        output.append(qc)
    return(output)

We can now implement step 2, encoding Alice's random bitlist into the random bases list

In [4]:
# Step 2
encoded_message = encode(alice_bits,alice_bases)
for i in range (4):
    print(encoded_message[i])
print(encoded_message[8])

     ┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             
     ┌───┐ ░ 
  q: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
             
     ┌───┐ ░ 
  q: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
             
     ┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             
     ┌───┐┌───┐ ░ 
  q: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════
                  


Display a couple of encoded qubits to check that everything is correct up to now

In **step 3**, Bob has to generate a set of random bases and measure each encoded qubit with it

In [5]:
# Step 3
bob_bases=randint(2, size=n)
print(bob_bases)

[1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1
 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1]


In [6]:
def measure(qubitlist,baseslist):
    backend = Aer.get_backend('aer_simulator')
    output = []
    for i in range(n):
        if baseslist[i] == 1:
            qubitlist[i].h(0)
        qubitlist[i].measure(0,0)
        
        aer_sim = Aer.get_backend('aer_simulator')
        result = aer_sim.run(qubitlist[i], shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        output.append(measured_bit)
    return output    

In [7]:
bob_bits = measure(encoded_message,bob_bases)

You can now display some of the messages to check that Bob has implemented the expected measure

In [8]:
print(bob_bits)

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0]


In **step 4**, Bob and Alice compare their respective bases list and discard all the bits whose corresponding bases are different

In [9]:
def purge(bitlist, baseslist1, baseslist2):
    output = []
    for i in range(n):
        if baseslist1[i]==baseslist2[i]:
            output.append(bitlist[i])
    return output
        

In [10]:
# step 4
alice_bits = purge(alice_bits,alice_bases,bob_bases)
bob_bits = purge(bob_bits,alice_bases,bob_bases)

print(*alice_bits)
print(*bob_bits)

0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 0 0
0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 0 0


Finally, in **step 5**, Alice and Bob choose and share a random selection of their remaining bits and compare them.
If enough of them are different, they discard the results. Else, the not compared bits are the common key

In [47]:
# Step 5

threshold = 0.05 # how much error we can accept
k = 25 # number of the selected bits to be completed
selection = choice(len(alice_bits), k, replace=False) #np.random.choice
selection.sort()
selection = selection[::-1]
error_rate = 0
for i in selection:
    b1 = alice_bits.pop(i)
    b2 = bob_bits.pop(i)
    if b1 != b2:
        error_rate = error_rate + 1/k
print("Error rate: ",error_rate)

if error_rate > threshold:
    print("Alert. Communication insecure")
else:
    print("Shared key: ",alice_bits)
    print("(it should coincide with ",bob_bits,")")

Error rate:  0
Shared key:  [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
(it should coincide with  [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1] )


Now, after completing the protocol and give it some tries...
Can you implement Eve eavesdropping, and check what happens?

In [ ]:
eve_bases=randint(2,size=n)
bob_message=randint(2,size=n)
#Eves has to do a full protocon between Alice and Bob
def alice_Eve_protocol(alice_bits,alice_bases,eve_bases):
    message=encode(alice_bits,alice_bases)
    decode_message=measure(message,eve_bases)
    ae_secret_key=purgue(decode_message,alice_bases,eve_bases)
    return ae_secret_key
    
def bob_Eve_protocol(bob_message,bob_bases,eve_bases):
    bob_message=encode(bob_bases,bob_bases)
    decode_message=measure(bob_message,eve_bases)
    be_secret_key=purgue(decode_message,bob_bases,eve_bases)
    return be_secret_key    

    
    
    

## Superdense coding


Alice wants to transmit two bits, say $ab$ to Bob, through a single qubit comminication channel

**-Step 1**

A $|\Phi^+\rangle$ state is shared between Alice and Bob.


**-Step 2**

If $a=1$, Alice applies $Z$ to her half of the entangled pair.

If $b=1$, Alice applies $X$ to her half of the entangled pair.

Then Alice sends her qubit to Bob.


**-Step 3**

Bob applies a CNOT (control in Alice and target in Bob). Then he applies a Hadamard in Bob's qubit and measures both.

a is recovered in Alice's qubit measure and b is recovered in Bob's.

Now it is yor turn to implement it...

In [ ]:
A = QuantumRegister(1,"A")
B = QuantumRegister(1,"B")

ar = QuantumRegister(1,"ar")
br = QuantumRegister(1,"br")

b = ClassicalRegister(1,"cb")
a = ClassicalRegister(1,"cb")

qcircuit=QuantumCircuit(A,B,ar,br,a,b)

qc.h(A)
qc.cnot(A,B)
qc.barrier()

qc.h(ar)
qc.h(br)
qc.barrier()

qc.measure(ar,a)
qc.measure(br,b)

with qc.if_test((a, 1)):
    qc.z(A)

with qc.if_test((b, 1)):
    qc.x(A)

qc.barrier()

qc.cnot(A,B)
qc.h(B)



